<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/symmetry1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.half().to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics
import math
import os
from torchvision import models, transforms
from PIL import Image
from sklearn.linear_model import RANSACRegressor
deeplab_model = models.segmentation.deeplabv3_resnet101(pretrained=True).to(device).eval()

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def symmetry_calc(left, right):
    si = abs(left - right) / (0.5 * (left + right))

    return {
        "symmetry": round(si, 3)
    }


def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def camera_to_pixel_coords(X, Y, Z, f_x, f_y, c_x, c_y):
    u = int((X * f_x) / Z + c_x)
    v = int((Y * f_y) / Z + c_y)
    return u, v

def point_to_plane_distance(point, plane_model):
    a, b, c, d = plane_model
    X, Y, Z = point
    return abs(a*X + b*Y + c*Z + d) / np.sqrt(a**2 + b**2 + c**2)

def processvideo(video_path, output_path,model):
    left=0
    right=0
    S_I=0
    vr = VideoReader(video_path)
    frame_count = len(vr)
    print(f"Total frames: {frame_count}")
    fps = 7
    step = fps * 1
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)
    mp_draw = mp.solutions.drawing_utils
    sample_frame = vr[0].asnumpy()
    rgb = cv2.cvtColor(sample_frame , cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    img1 = transform(rgb).unsqueeze(0).to(device).half()
    height, width, _ = sample_frame.shape
    c_x,c_y= approximate_intrinsics(width, height)
    with torch.no_grad():
            pred = model.infer(img1)
            depth_map = pred["depth"].squeeze().cpu().numpy()
            depth_map = cv2.resize(depth_map, (sample_frame.shape[1], sample_frame.shape[0]))
            intrinsics = pred["focallength_px"].cpu().numpy()
            if np.ndim(intrinsics) == 0:
              fx = fy = float(intrinsics)
            else:
              fx, fy = intrinsics
    print("kk")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for i in range(0, frame_count, step):
        frame = vr[i].asnumpy()
        rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        #ectracting ground
        img = transform(rgb).unsqueeze(0).to(device)
        deeplab_input = preprocess(rgb).unsqueeze(0).to(device)

        with torch.no_grad():
           out1 = deeplab_model(deeplab_input)['out'][0]
           seg = out1.argmax(0).cpu().numpy()

        ground_mask = (seg == 0) | (seg == 2)

        cv2.imwrite("ground_mask.png", ground_mask.astype(np.uint8) * 255)
        h1, w1 = depth_map.shape
        ys, xs = np.where(ground_mask)

        ground_points = []
        for u, v in zip(xs, ys):
            Z = depth_map[v, u]
            if Z > 0:  # valid depth
                   X, Y, Z = pixel_to_camera_coords(u, v, Z, fx, fy, c_x, c_y)
                   ground_points.append([X, Y, Z])

        ground_points = np.array(ground_points)
        XY = ground_points[:, :2]
        Z = ground_points[:, 2]
        ransac = RANSACRegressor(residual_threshold=0.02)
        ransac.fit(XY, Z)
        a, b = ransac.estimator_.coef_
        c = ransac.estimator_.intercept_
        plane = [a, b, -1, c]

        points = {}
        mp_pose = mp.solutions.pose
        h, w, _ = frame.shape
        if results.pose_landmarks:
           for id, lm in enumerate(results.pose_landmarks.landmark):
             cx= min(max(int(lm.x * w), 0), w - 1)
             cy= min(max(int(lm.y * h), 0), h - 1)
             cz= depth_map[cy, cx]

             X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
             if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
                points["left_ankle"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
                points["right_ankle"] = (X, Y, Z)
           distl = point_to_plane_distance(points["left_ankle"], plane)
           distr= point_to_plane_distance(points["right_ankle"], plane)
           print(f"distl: {distl}")
           print(f"distr: {distr}")
           cv2.putText(frame,
            f"Frame: {i} | left: {distl:.4f} | right: {distr:.4f}",
            (10, 30),  # (x,y) for first line
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,       # bigger font
            (0, 0, 200),
            2,
            cv2.LINE_AA)

           if distl < 0.005 and distr >= 0.005:
            print(distl)
            left= left+1
           elif distr < 0.005 and distl >= 0.005:
            print(distr)
            right=right+1

        out.write(frame)
    if left > 0 or right !=0:
       S_I= symmetry_calc(left,right)
       print(S_I)
    black_frame = np.zeros_like(frame)
    for _ in range(fps * 3):

         frame_with_text = black_frame.copy()

         txt = f"symmetry: {S_I}"

         font = cv2.FONT_HERSHEY_SIMPLEX
         font_scale = 2.0
         thickness = 3
         color = (255, 255, 255)


         (text_width, text_height), baseline = cv2.getTextSize(txt, font, font_scale, thickness)


         x = (width - text_width) // 2
         y = (height + text_height) // 2


         cv2.putText(frame_with_text,
            txt,
            (x, y),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)

         out.write(frame_with_text)
    out.release()


input_folder = "/content/drive/MyDrive/test"
output_folder = "/content/drive/MyDrive/testr"
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".mp4", ".avi", ".mov"))]

for video in video_files:
    input_path = os.path.join(input_folder, video)
    output_name = os.path.splitext(video)[0] + "_fin.mp4"
    output_path = os.path.join(output_folder, output_name)

    print(f"Processing {video} ...")
    processvideo(input_path, output_path,model)
    print(f"Saved result to {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing Recording 2025-09-08 211900 - Trimp.mp4 ...
Total frames: 164
kk
distl: 0.008439491459451796
distr: 0.007134797823680668
distl: 0.002242460471654648
distr: 0.006171361895128113
0.002242460471654648
distl: 0.0014932568615450474
distr: 0.003520503217486819
distl: 0.004777719360896644
distr: 0.004211415581438739
distl: 0.017999506079185244
distr: 0.01127856267486098
distl: 0.003672492633756326
distr: 0.0031295720472643393
distl: 0.0009371571157869296
distr: 0.0008583702535579823
distl: 0.0031383714739603163
distr: 0.007537398780691678
0.0031383714739603163
distl: 0.0010479444149333397
distr: 0.001743373620743513
distl: 0.015025179265791723
distr: 0.017176570324867243
distl: 0.0017767665564249677
distr: 0.001934046627036295
distl: 0.010252631735218577
distr: 0.0023473088621726464
0.0023473088621726464
{'symmetry': 0.667}
Saved result to /content/drive/MyDrive/testr/Recording 2025-09-08 211900 - Trimp_fin.mp4


In [ ]:
!pip install mediapipe
!pip install decord


!pip install git+https://github.com/apple/ml-depth-pro.git
!mkdir -p checkpoints
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O checkpoints/depth_pro.pt

  Cloning https://github.com/apple/ml-depth-pro.git to /tmp/pip-req-build-in855b1x
  Running command git clone --filter=blob:none --quiet https://github.com/apple/ml-depth-pro.git /tmp/pip-req-build-in855b1x
  Resolved https://github.com/apple/ml-depth-pro.git to commit 9efe5c1def37a26c5367a71df664b18e1306c708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.4 MB/s eta 0:00:00
  Created wheel for depth_pro: filename=depth_pro-0.1-py3-none-any.whl size=27593 sha256=2535247aafa453bd87b4d0e87e27dcf878d6936095bcb545a8a2ff353518fde6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ir7blkd/wheels/3f/ad/4e/a2b790772df19931cab8bf9c5af51665ae6502f39589e662c4
Successfully built depth_pro
--2025-09-09 14:23:36--  https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt
Resolving huggingface.co (huggingface.co)... 52.84.217.103, 52.84.217.69, 52.8